# 案例 2.2: 全流域水资源联合调度

本案例演示了一个包含多个智能体和一个拥有紧急否决权的中央调度中心的综合性流域调度场景。它旨在模拟在洪水和高峰用水需求同时发生时的复杂决策过程。

### 1. 导入库并加载配置

In [ ]:
import json
import math

# 导入平台组件
from swp.central_coordination.collaboration.message_bus import MessageBus
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.simulation_identification.physical_objects.river_channel import RiverChannel
from swp.local_agents.control.hydropower_station_agent import HydropowerStationAgent
from swp.local_agents.disturbances.water_use_agent import WaterUseAgent
from swp.central_coordination.dispatch.central_dispatcher_agent import CentralDispatcherAgent

CONFIG_PATH = 'examples/integrated_basin_scheduling.json'
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置加载成功！")

### 2. 定义仿真函数

In [ ]:
def run_basin_simulation(cfg):
    # 1. 初始化
    bus = MessageBus()
    sim_cfg = cfg['simulation']
    dt = sim_cfg['dt']
    comp_cfgs = cfg['components']
    agent_cfgs = cfg['agents']

    # 2. 创建物理组件
    res_cfg = comp_cfgs['reservoir']
    initial_level = res_cfg['initial_volume'] / res_cfg['surface_area']
    reservoir = Reservoir("main_reservoir", {'volume': res_cfg['initial_volume'], 'water_level': initial_level}, {'surface_area': res_cfg['surface_area']})

    tg_cfg = comp_cfgs['turbine_gate']
    turbine_gate = Gate("turbine_gate", {'opening': 0}, tg_cfg, bus, "station/tg1")

    fg_cfg = comp_cfgs['flood_gate']
    flood_gate = Gate("flood_gate", {'opening': 0}, fg_cfg, bus, "station/flood")

    rc_cfg = comp_cfgs['river_channel']
    channel = RiverChannel("main_river", {'volume': rc_cfg['initial_volume'], 'outflow': rc_cfg['initial_outflow']}, {'k': rc_cfg['k']})

    sg_cfg = comp_cfgs['supply_gate']
    supply_gate = Gate("supply_gate", {'opening': 0}, sg_cfg, bus, "downstream/supply")

    # 3. 创建智能体
    hp_agent = HydropowerStationAgent("HydropowerOps", bus, reservoir, "station/flood", "station/supply", ["station/tg1"], agent_cfgs['hydropower_agent'])
    user_agent = WaterUseAgent("CityWaterDept", bus, supply_gate_topic="downstream/supply")
    dispatcher = CentralDispatcherAgent("BasinControlCenter", bus, reservoir, "downstream/supply", agent_cfgs['dispatcher_agent'])

    # 4. 仿真循环
    print("\n--- 开始仿真: 洪水与高峰用水并发场景 ---")
    dist_cfg = cfg['disturbance']
    base_inflow = dist_cfg['base_inflow']
    pulse_cfg = dist_cfg['flood_pulse']

    for t in range(sim_cfg['num_steps']):
        current_time = t * dt
        
        # 计算洪水扰动
        inflow = base_inflow
        if pulse_cfg['start_hour'] <= t < pulse_cfg['start_hour'] + pulse_cfg['duration_hours']:
            inflow += pulse_cfg['magnitude'] * math.sin((t - pulse_cfg['start_hour']) / pulse_cfg['duration_hours'] * math.pi)

        print(f"\n--- 小时 {t}, 来水: {inflow:.2f} m^3/s, 水库水位: {reservoir.get_state()['water_level']:.2f}m ---")

        # 5. 智能体决策 (感知 -> 本地行动 -> 全局覆盖)
        hp_agent.run(current_time)
        user_agent.run(current_time)
        dispatcher.run(current_time)

        # 6. 物理状态更新
        res_level = reservoir.get_state()['water_level']
        turbine_gate.step({'upstream_head': res_level, 'downstream_head': 10}, dt)
        flood_gate.step({'upstream_head': res_level, 'downstream_head': 10}, dt)
        station_outflow = turbine_gate.get_state()['outflow'] + flood_gate.get_state()['outflow']

        reservoir.set_inflow(inflow)
        reservoir.step({'outflow': station_outflow}, dt)

        channel.set_inflow(station_outflow)
        channel.step({}, dt)

        channel_head = channel.get_state()['volume'] * rc_cfg['k']
        supply_gate.step({'upstream_head': channel_head, 'downstream_head': 0}, dt)

        channel.set_state({'volume': channel.get_state()['volume'] - supply_gate.get_state()['outflow'] * dt})

        # 7. 日志记录
        print(f"  水电站决策: 发电闸开度={turbine_gate.get_state()['opening']:.2f}, 泄洪闸开度={flood_gate.get_state()['opening']:.2f}")
        print(f"  城市用水决策: 供水闸开度 -> {supply_gate.get_state()['opening']:.2f}")
        print(f"  调度中心状态: 监控中. 紧急水位: {dispatcher.emergency_flood_level}m")
        print(f"  河道出流: {channel.get_state()['outflow']:.2f} m^3/s")

### 3. 运行仿真

In [ ]:
run_basin_simulation(config)